In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
from datetime import datetime

## Creating a Database

In [ ]:
#Creating a postgresql database

#db connection
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "postgres")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''CREATE DATABASE stock_data'''

cursor.execute(sql)
connection.close()


## Cleaning DataFrames for PostgreSQL Compatibility

In [ ]:
#Option Chain Table
def get_options_chain_df(ticker):
    ticker_object = yf.Ticker(ticker)

    #Expirations Dates
    expiration_dates = ticker_object.options
    
    #Chain data for each expiration date
    options = pd.DataFrame()
    for each_date in expiration_dates:
        option_data = ticker_object.option_chain(each_date)
        option_data = pd.DataFrame().append(option_data.calls).append(option_data.puts)
        options = options.append(option_data, ignore_index=True)

    #Cleaning the dataframe for PostgreSQL compatibility                 
    return (options
                .assign(
                        ticker = ticker,
                        inTheMoney = options['inTheMoney'].astype(float),
                        isCall = options['contractSymbol'].str[4:].apply(lambda x: "C" in x).astype(float),
                        bid = options['bid'].apply(pd.to_numeric),
                        ask = options['ask'].apply(pd.to_numeric),
                        strike = options['strike'].apply(pd.to_numeric),
                        mark_midpoint_of_bid_ask = (options.bid + options.ask)/2,
                        expirationDate = pd.to_datetime(options['contractSymbol'].str[6:8] + '-' + options['contractSymbol'].str[8:10] + '-' + '20' + options['contractSymbol'].str[4:6]).dt.date
                        )
                .drop(columns=['contractSize','currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])
                .dropna()
                .to_records(index = False)
            )  


#Historical Stock Prices Table
def get_hist_df(ticker):
    df = yf.download(ticker, period = 'max')
    return (df
              .rename(columns = {'Adj Close':'Adj_Close'})
              .assign(
                        Date = np.datetime_as_string(df.index, unit = 'D'),
                        Ticker = ticker)
              .set_index('Date')

              #PostgreSQL requires df to be inserted as records
              .to_records(index = True)
            )


#Info Table
def get_info_df(ticker):
    
    #Creating a dataframe from non-compatible dictionary
    df = pd.DataFrame(list(yf.Ticker(ticker).info.items()))
    
    #Keeping certain columns
    cols = ['zip','sector','city','state','country','industry','ticker']

    #Cleaning the dataframe for PostgreSQL compatibility
    return (df
                .transpose()
                .rename(columns=df.transpose().iloc[0])
                .iloc[1:]
                .assign(
                    ticker = ticker)
                .reindex(columns = cols)
                .to_records(index = False)
                )
                

#Institutional Ownership Table
def get_instit_ownership_df(ticker):
    df = yf.Ticker(ticker).institutional_holders
    return (df
                .rename(columns = (lambda x: x.replace(' ','_')))
                .assign(
                    ticker = ticker,
                    Date_Reported = df['Date Reported'].dt.date,
                    percent_outstanding_shares = df['% Out']*100
                    )
                .drop(columns= '%_Out')
                .to_records(index = False)
            )


#Balance Sheet Table
def get_balance_sheet_df(ticker):
    df = yf.Ticker(ticker).balance_sheet

    return (df
                .transpose()
                .assign(
                    Date = np.datetime_as_string(df.transpose().index, unit = 'D'),
                    ticker = ticker)
                .reset_index(drop = True)
                .rename(columns = lambda x: x.replace(' ','_'))
                .to_records(index = False)
            )

#Cash Flow Table
def  get_cash_flow_df(ticker):
    df = yf.Ticker(ticker).cashflow
    return (df
                .transpose()
                .rename(columns=lambda x: x.replace(' ','_'))
                .assign(
                    ticker = ticker,
                    Date = np.datetime_as_string(df.transpose().index, unit = 'D'),
                    Free_Cash_Flow =  (df.transpose()['Total Cash From Operating Activities']) + (df.transpose()['Capital Expenditures']))
                .set_index('Date')
                .to_records(index = True)
            )


#Income Statement
def get_income_statement_df(ticker):
    df = yf.Ticker(ticker).financials
    return (df
                .transpose()
                .drop(columns = ['Non Recurring','Effect Of Accounting Charges','Minority Interest', 'Discontinued Operations','Other Items','Other Operating Expenses','Extraordinary Items'])
                .rename(columns= lambda x: x.replace(' ','_'))
                .assign(
                    ticker = ticker,
                    Date = np.datetime_as_string(df.transpose().index, unit = 'D'),
                    Operating_Expenses = (df.transpose()['Selling General Administrative']) + (df.transpose()['Research Development'])
                    )
                .reset_index(drop=True)
                .to_records(index = False)
            )


#Firm Recommendations Table
def get_recommendations_df(ticker):
    df = yf.Ticker(ticker).recommendations
    return (df
                .reset_index()
                .rename(columns=lambda x:x.replace(' ','_'))
                .assign(
                    Date = np.datetime_as_string(df.index, unit = 'D'),
                    ticker = ticker
                )
                .to_records(index = False)
            )
    

#Earnings Calendar and Concensus Revenue/EPS Expectation Table
def get_earnings_expectations_df(ticker):
    df = yf.Ticker(ticker).calendar.transpose().head(1)
    return (df
                .rename(columns = lambda x: x.replace(' ','_'))
                .assign(
                    Earnings_Date = df['Earnings Date'].dt.date,
                    ticker = ticker
                )
                .to_records(index = False)
            )


#ESG Table - Sustainability metrics provided by Sustainalytics
def get_esg_df(ticker):
    #Keeping specific list of columns
    cols = ['socialScore','governanceScore','environmentScore','peerGroup','percentile','esgPerformance']
    
    #Create dataframe for companies reporting ESG metrics
    try:
        df = yf.Ticker(ticker).sustainability
        return (df
                    .transpose()
                    .reindex(columns = cols)
                    .assign(
                        ticker = ticker
                    )
                    .to_records(index = False)
                )

    #Create dataframe for companies not reporting ESG metrics                
    except:
        
        #List of strings
        empty_list = []
        for col in cols:
            empty_list.append('No ESG value provided')
        
        #Merging lists and creating a dataframe
        d = {'list2':cols,"list1":empty_list}
        df = pd.DataFrame(d)
        return (df
                        .transpose()
                        .rename(columns = df.transpose().iloc[0])
                        .iloc[1:]
                        .assign(
                            ticker = ticker)
                        .reset_index(drop = True)
                        .to_records(index = False)
                )

## ESG TABLE

In [ ]:
#Create ESG Table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS ESG_Data;       
        CREATE TABLE ESG_Data
               (
                socialScore VARCHAR(255) NOT NULL,
                governanceScore VARCHAR(255) NOT NULL,
                environmentScore VARCHAR(255) NOT NULL,
                peerGroup VARCHAR(255) NOT NULL,
                percentile VARCHAR(255) NOT NULL,
                esgPerformance VARCHAR(255) NOT NULL,
                ticker VARCHAR(255) NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Insert ESG Data
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO ESG_Data (socialScore,governanceScore,environmentScore,peerGroup,percentile,esgPerformance,ticker)
               VALUES (%s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_esg_df('PLTR'))
connection.close()


## Earnings Expectations Table

In [ ]:
#Create Earnings Expectations Table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS Earnings_Expectations;       
        CREATE TABLE Earnings_Expectations
               (
                Earnings_Date DATE NOT NULL,
                ticker VARCHAR(255) NOT NULL,
                Earnings_Average FLOAT NOT NULL,
                Earnings_Low FLOAT NOT NULL,
                Earnings_High FLOAT NOT NULL,
                Revenue_Average FLOAT NOT NULL,
                Revenue_Low FLOAT NOT NULL,
                Revenue_High FLOAT NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Insert Earnings Expectations Date
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO Earnings_Expectations (Earnings_Date, ticker, Earnings_Average, Earnings_Low, Earnings_High, Revenue_Average, Revenue_Low, Revenue_High)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_earnings_expectations_df('AAPL'))
connection.close()

## Recommendations Table

In [ ]:
#Create Recommendations Table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS Analyst_Recommendations;       
        CREATE TABLE Analyst_Recommendations
               (
                Date DATE NOT NULL,
                FIRM VARCHAR(255) NOT NULL,
                To_Grade VARCHAR(255) NOT NULL,
                From_Grade VARCHAR(255) NOT NULL,
                Action VARCHAR(255) NOT NULL,
                ticker VARCHAR(255) NOT NUll
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Inserting Analyst Recommendations Data
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO Analyst_Recommendations (Date, Firm, To_Grade, From_Grade, Action, ticker)
               VALUES (%s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_recommendations_df('AAPL'))
connection.close()


## Income Statement Table

In [ ]:
#Create Income Statement Table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS income_statement;       
        CREATE TABLE income_statement
               (
                Research_Development FLOAT NOT NULL,
                Income_Before_Tax FLOAT NOT NULL,
                Net_Income FLOAT NOT NULL,
                Selling_General_Administrative FLOAT NOT NULL,
                Gross_Profit FLOAT NOT NULL,
                Ebit FLOAT NOT NULL,
                Operating_Income FLOAT NOT NULL,
                Interest_Expense FLOAT NOT NULL,
                Income_Tax_Expense FLOAT NOT NULL,
                Total_Revenue FLOAT NOT NULL,
                Total_Operating_Expenses FLOAT NOT NULL,
                Cost_Of_Revenue FLOAT NOT NULL,
                Total_Other_Income_Expense_Net FLOAT NOT NULL,
                Net_Income_From_Continuing_Ops FLOAT NOT NULL,
                Net_Income_Applicable_To_Common_Shares FLOAT NOT NULL,
                ticker VARCHAR(255) NOT NULL,
                Date DATE NOT NULL,
                Operating_Expenses FLOAT NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Insert Income Statement Data
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO income_statement (Research_Development,Income_Before_Tax,Net_Income,Selling_General_Administrative,Gross_Profit,Ebit,Operating_Income,
                                        Interest_Expense,Income_Tax_Expense,Total_Revenue,Total_Operating_Expenses,Cost_Of_Revenue,Total_Other_Income_Expense_Net,
                                        Net_Income_From_Continuing_Ops,Net_Income_Applicable_To_Common_Shares,ticker,Date,Operating_Expenses)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_income_statement_df('AAPL'))
connection.close()

## Balance Sheet Table

In [ ]:
#Creating balance sheet table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS balance_sheet;       
        CREATE TABLE balance_sheet
               (
                Total_Liab FLOAT NOT NULL,
                Total_Stockholder_Equity FLOAT NOT NULL,
                Other_Current_Liab FLOAT NOT NULL,
                Total_Assets FLOAT NOT NULL,
                Common_Stock FLOAT NOT NULL,
                Other_Current_Assets FLOAT NOT NULL,
                Retained_Earnings FLOAT NOT NULL,
                Other_Liab FLOAT NOT NULL,
                Treasury_Stock FLOAT NOT NULL,
                Other_Assets FLOAT NOT NULL,
                Cash FLOAT NOT NULL,
                Total_Current_Liabilities FLOAT NOT NULL,
                Short_Long_Term_Debt FLOAT NOT NULL,
                Other_Stockholder_Equity FLOAT NOT NULL,
                Property_Plant_Equipment FLOAT NOT NULL,
                Total_Current_Assets FLOAT NOT NULL,
                Long_Term_Investments FLOAT NOT NULL,
                Net_Tangible_Assets FLOAT NOT NULL,
                Short_Term_Investments FLOAT NOT NULL,
                Net_Receivables FLOAT NOT NULL,
                Long_Term_Debt FLOAT NOT NULL,
                Inventory FLOAT NOT NULL,
                Accounts_Payable FLOAT NOT NULL,
                Date DATE NOT NULL,
                ticker VARCHAR(255) NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Inserting balance sheet data
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO balance_sheet (Total_Liab, Total_Stockholder_Equity, Other_Current_Liab,
       Total_Assets, Common_Stock, Other_Current_Assets,
       Retained_Earnings, Other_Liab, Treasury_Stock, Other_Assets,
       Cash, Total_Current_Liabilities, Short_Long_Term_Debt,
       Other_Stockholder_Equity, Property_Plant_Equipment,
       Total_Current_Assets, Long_Term_Investments, Net_Tangible_Assets,
       Short_Term_Investments, Net_Receivables, Long_Term_Debt,
       Inventory, Accounts_Payable, Date, ticker)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_balance_sheet_df('AAPL'))
connection.close()

## Institutional Ownership Table

In [ ]:
#Creating institutional ownership table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS instit_owners;       
        CREATE TABLE instit_owners
               (
                Holder VARCHAR(255) NOT NULL,
                Shares FLOAT NOT NULL,
                Date_Reported DATE NOT NULL,
                Value FLOAT NOT NULL,
                ticker VARCHAR(255) NOT NULL,
                perc_outstanding_shares FLOAT NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Inserting instit ownership data
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO instit_owners (Holder, Shares, Date_Reported, Value, ticker, perc_outstanding_shares)
               VALUES (%s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_instit_ownership_df('AAPL'))
connection.close()

## Info Table

In [ ]:
#Creating info table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS info_table;       
        CREATE TABLE info_table
               (
                zip INT NOT NULL,
                sector VARCHAR(255) NOT NULL,
                city VARCHAR(255) NOT NULL,
                state VARCHAR(255) NOT NULL,
                country VARCHAR(255) NOT NULL,
                industry VARCHAR(255) NOT NULL,
                ticker VARCHAR(255) NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Inserting info data
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO info_table (zip, sector, city, state, country, industry, ticker)
               VALUES (%s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_info_df("AAPL"))
connection.close()

## Option Chains Table

In [ ]:
#Creating a options_chain table 
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS option_chains;       
        CREATE TABLE option_chains
               (
                contractSymbol VARCHAR(255) NOT NULL,
                strike FLOAT NOT NULL,
                bid FLOAT NOT NULL,
                ask FLOAT NOT NULL,
                volume INT NOT NULL,
                openinterest FLOAT NOT NULL,
                impliedvolatility FLOAT NOT NULL,
                inthemoney FLOAT NOT NULL,
                ticker VARCHAR(255) NOT NULL,
                iscall FLOAT NOT NULL,
                mark_midpoint_of_bid_ask FLOAT NOT NULL,
                expirationDate DATE NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Inserting data into new table options_chains
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO option_chains (contractSymbol, strike, bid, ask, volume, openinterest, impliedvolatility, inthemoney, ticker, iscall, mark_midpoint_of_bid_ask, expirationDate)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''

cursor.executemany(sql, get_options_chain_df('AAPL'))
connection.close()


## Historical Prices Table

In [ ]:
#Historical Stock Pricing Table
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''       DROP TABLE IF EXISTS historical_prices;
                CREATE TABLE historical_prices
               (
                Date DATE NOT NULL,
                Open FLOAT NOT NULL,
                High FLOAT NOT NULL,
                Low FLOAT NOT NULL,
                Close FLOAT NOT NULL,
                Adj_Close FLOAT NOT NULL,
                Volume BIGINT NOT NULL,
                Ticker VARCHAR(255) NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Inserting data into new table historical_prices

connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO historical_prices (Date, Open, High, Low, Close, Adj_Close, Volume, Ticker)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(sql, get_hist_df("AAPL"))
connection.close()

## Cash Flow Table

In [ ]:
#Creating a cash_flow table 
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''
        DROP TABLE IF EXISTS cash_flows;       
        CREATE TABLE cash_flows
               (
                Year DATE NOT NULL,
                Investments FLOAT NOT NULL,
                Change_To_Liabilities FLOAT NOT NULL,
                Total_Cashflows_From_Investing_Activities FLOAT NOT NULL,
                Net_Borrowings FLOAT NOT NULL,
                Total_Cash_From_Financing_Activities FLOAT NOT NULL,
                Change_To_Operating_Activities FLOAT NOT NULL,
                Issuance_Of_Stock FLOAT NOT NULL,
                Net_Income FLOAT NOT NULL,
                Change_In_Cash FLOAT NOT NULL,
                Repurchase_Of_Stock FLOAT NOT NULL,
                Total_Cash_From_Operating_Activities FLOAT NOT NULL,
                Depreciation FLOAT NOT NULL,
                Other_Cashflows_From_Investing_Activities FLOAT NOT NULL,
                Dividends_Paid FLOAT NOT NULL,
                Change_To_Inventory FLOAT NOT NULL,
                Change_To_Account_Receivables FLOAT NOT NULL,
                Other_Cashflows_From_Financing_Activities FLOAT NOT NULL,
                Change_To_Netincome FLOAT NOT NULL,
                Capital_Expenditures FLOAT NOT NULL,
                ticker VARCHAR(255) NOT NULL,
                Free_Cash_Flow FLOAT NOT NULL
                );'''

cursor.execute(sql)
connection.close()

In [ ]:
#Inserting data into new table cash_flow
connection = psycopg2.connect(
        user = "postgres",
        password = "password",
        host = "localhost",
        port = "5432",
        database = "stock_data")

connection.autocommit = True
cursor = connection.cursor()

#SQL Code Block
sql = '''INSERT INTO cash_flows (Year, Investments,Change_To_Liabilities,Total_Cashflows_From_Investing_Activities, Net_Borrowings,Total_Cash_From_Financing_Activities,Change_To_Operating_Activities,Issuance_Of_Stock,Net_Income,Change_In_Cash,Repurchase_Of_Stock,Total_Cash_From_Operating_Activities,Depreciation,Other_Cashflows_From_Investing_Activities,Dividends_Paid,Change_To_Inventory,Change_To_Account_Receivables,Other_Cashflows_From_Financing_Activities,Change_To_Netincome,Capital_Expenditures,ticker, Free_Cash_Flow)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''

cursor.executemany(sql, get_cash_flow_df('AAPL'))
connection.close()


## Test Query

In [ ]:
#Testing query
connection = psycopg2.connect(
    database='stock_data',
    user='postgres', 
    password='password', 
    host='localhost', 
    port= '5432'
)

connection.autocommit = True
cursor = connection.cursor()

sql = """SELECT * from historical_prices LIMIT 5"""

cursor.execute(sql)
rows = cursor.fetchall()
for row in rows:
    print(row)

connection.close()